In this code example, we'll apply LQR feedback control to stabilize a 2D quadrotor at a point.

In [1]:
%pylab inline
from pylab import *

import control
import matplotlib.animation
import scipy.integrate

rc("animation", html="html5")

import path_utils
path_utils.add_relative_to_current_source_file_path_to_sys_path("../../lib")

import flashlight.curve_utils       as curve_utils
import flashlight.quadrotor_2d      as quadrotor_2d
import flashlight.spline_utils      as spline_utils
import flashlight.trig_utils        as trig_utils

Populating the interactive namespace from numpy and matplotlib
Initializing flashlight v0.0.1
flashlight.quadrotor_2d: Constructing sympy symbols...
flashlight.quadrotor_2d: Finished constructing sympy symbols (0.004 seconds).
flashlight.quadrotor_2d: Loading sympy modules...
flashlight.quadrotor_2d: Finished loading sympy modules (0.004 seconds).


/Users/mike/Code/GitHub/flashlight/code/lib/flashlight/transformations.py:1888: UserWarning: failed to import module _transformations
  warnings.warn("failed to import module %s" % name)


Applying LQR feedback control to quadrotors requires that we linearize the quadrotor dynamics. We begin by defining the point in state-control space about which we want to linearize. In this example, we choose to linearize the dynamics about hovering-at-the-origin. To compute the control forces required for hovering, we must use the mass and gravitational acceleration constants assumed by our quadrotor dynamics model. We access these constants using the `quadrotor_2d.m` and `quadrotor_2d.g` variables. These variables can also be modified to change the mass and gravity of the quadrotor assumed in the `quadrotor_2d` module. Note that the flashlight conventions for ordering the entries in our state and control vectors are as follows.

```
x = matrix( [ p_y, p_x, theta, p_y_dot, p_x_dot, theta_dot ] ).T
u = matrix( [ u_left, u_right ] ).T
```

To linearize the quadrotor dynamics, we call `quadrotor_2d.compute_df_dx_and_df_du` and pass in our linearization point. This function will return two matrices `A` and `B` that define the linearized dynamics. Next, we must also define two cost matrices `Q` and `R` that specify how we penalize deviations away from our goal of hovering-at-the-origin.

After having defined all of these matrices, we can compute a control matrix `K`, such that the optimal control force `u = -K*x` for all states `x`. We rely on the `control` library, which is not a part of flashlight, to compute `K`. For more details on the underlying optimal control problem being solved here, see [1].

#### References

[1] Russ Tedrake. Underactuated Robotics: Algorithms for Walking, Running, Swimming, Flying, and Manipulation (course notes for MIT 6.832). 2016. http://underactuated.mit.edu

In [2]:
m = quadrotor_2d.m
g = quadrotor_2d.g

x_star = matrix([0,0,0,0,0,0]).T
u_star = matrix([m*g/2.0,m*g/2.0]).T

Q = diag([1,1,1,1,1,1])
R = diag([1,1])

A, B    = quadrotor_2d.compute_df_dx_and_df_du(x_star, u_star)
K, S, E = control.lqr(A, B, Q, R)

print "A ="; print A; print; print "B ="; print B; print; print "K ="; print K

A =
[[ 0.   0.   0.   1.   0.   0. ]
 [ 0.   0.   0.   0.   1.   0. ]
 [ 0.   0.   0.   0.   0.   1. ]
 [ 0.   0.  -0.   0.   0.   0. ]
 [ 0.   0.  -9.8  0.   0.   0. ]
 [ 0.   0.   0.   0.   0.   0. ]]

B =
[[ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 1.  1.]
 [-0. -0.]
 [-1.  1.]]

K =
[[ 0.70710678  0.70710678 -8.34815843  1.09868411  1.30564234 -2.97458542]
 [ 0.70710678 -0.70710678  8.34815843  1.09868411 -1.30564234  2.97458542]]


Next, we set up our simulation callback function to compute the appropriate feedback control forces and compute the time derivative of our quadrotor state. When computing the matrix `K`, the `control` library assumes that our goal is to get back to the origin of state-control space. However, our goal is to hover (i.e., apply nonzero control) at the origin of state space. To handle this more general control problem, we simply translate our coordinate system, so that our intended goal point is at the origin of our translated coordinate system. For more details on this strategy, see [1].

Strictly speaking, we should include a bit of extra logic here to handle the fact that the `theta` dimension of our state space wraps around. But we omit this detail for simplicity.

Finally, we specify the initial state of our simulation to have a large initial velocity, to simulate a large unexpected disturbance (e.g., a gust of wind) occuring at the beginning of the simulation.

#### References

[1] Russ Tedrake. Underactuated Robotics: Algorithms for Walking, Running, Swimming, Flying, and Manipulation (course notes for MIT 6.832). 2016. http://underactuated.mit.edu

In [3]:
x_disturbance = matrix([0.0, 0.0, 0.0, 5.0, 5.0, -4.0*pi]).T
x_0           = (x_star + x_disturbance).A1

def compute_x_dot(x_t, t):

    x_t     = matrix(x_t).T    
    x_bar_t = x_t - x_star
    u_bar_t = -K*x_bar_t
    u_t     = u_bar_t + u_star
    x_dot_t = quadrotor_2d.compute_x_dot(x_t, u_t).A1
    
    return x_dot_t

num_timesteps_sim = 200

t_begin = 0.0
t_end   = 10.0
t_sim   = linspace(t_begin, t_end, num_timesteps_sim)
x_sim   = scipy.integrate.odeint(compute_x_dot, x_0, t_sim)

figsize(6,4)
quadrotor_2d.draw(t_sim, x_sim, inline=True)